# PART 1 -------------------------------

## Libs Import

In [5]:

import warnings
warnings.filterwarnings("ignore") 

import os
import math
from functools import partial

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import timm
from timm.data.transforms_factory import create_transform
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, confusion_matrix
import lightgbm as lgb

from tqdm import tqdm

import glob


## Parameters

In [24]:
work_dir = './data'

train_file_path = f"{work_dir}/Train.csv"
test_file_path = f"{work_dir}/Test.csv"


FEAT_PATH_ = f'{work_dir}/Features_build'

SEED_ = 42

v1_models = [
    ('efficientnet_b3.ra2_in1k',224), 
    ('resnet34',224),
    ('vit_base_patch16_clip_224.openai', 224),
    ('resnet50',224),
    ('vit_large_patch14_reg4_dinov2.lvd142m',518),
]

v3_models = [
    ('efficientnet_b3.ra2_in1k',224), 
    ('resnet34',224),
    ('vit_base_patch16_clip_224.openai', 224),
]


# 1. Work On Images NZP File Images

In [25]:
## The Start of Process & Usage of Images ## 

## Features Extractions Functions & Pipelines

In [26]:

# =============================================================================
# Function For Transforming V1 & Extraction
# =============================================================================


# =============================================================================
# Constants and Basic Helper Functions
# =============================================================================
_MAX_INT = np.iinfo(np.uint16).max

def decode_slope(x: np.ndarray) -> np.ndarray:
    """Convert 16-bit discretized slope to float32 radians."""
    return (x / _MAX_INT * (math.pi / 2.0)).astype(np.float32)

def normalize(x: np.ndarray, mean: int, std: int) -> np.ndarray:
    """Normalize using (x - mean)/std."""
    return (x - mean) / std

rough_S2_normalize = partial(normalize, mean=1250, std=500)

def preprocess_image(x: np.ndarray) -> np.ndarray:
    """
    Preprocess composite image:
      - If the image has a time series (ndim==4), average over time.
      - Normalize the first five Sentinel‑2 bands and decode the slope.
    Assumes channels are:
      0: B2 (Blue), 1: B3 (Green), 2: B4 (Red), 3: B8 (NIR),
      4: B11 (SWIR), 5: slope.
    """
    if x.ndim == 4:
        x = x.mean(axis=0)
    processed = np.concatenate([
        rough_S2_normalize(x[..., :-1].astype(np.float32)),
        decode_slope(x[..., -1:])
    ], axis=-1).astype(np.float32)
    return processed

def compute_band_stats(image: np.ndarray) -> dict:
    """Compute per-band mean and standard deviation."""
    stats = {}
    for ch in range(image.shape[-1]):
        stats[f'band{ch}_mean'] = float(np.nanmean(image[..., ch]))
        stats[f'band{ch}_std'] = float(np.nanstd(image[..., ch]))
    return stats

def compute_indices(image: np.ndarray) -> dict:
    """
    Compute NDVI and NDMI (here called endmi) indices.
    For the image:
      - Red (B4) is channel 2
      - NIR (B8) is channel 3
      - SWIR (B11) is channel 4
    """
    B4 = image[..., 2]
    B8 = image[..., 3]
    B11 = image[..., 4]
    ndvi_den = (B8 + B4)
    ndvi = np.where(ndvi_den != 0, (B8 - B4) / ndvi_den, 0)
    ndmi_den = (B8 + B11)
    ndmi = np.where(ndmi_den != 0, (B8 - B11) / ndmi_den, 0)
    return {
        'ndvi_mean': float(np.nanmean(ndvi)),
        'ndvi_std': float(np.nanstd(ndvi)),
        'endmi_mean': float(np.nanmean(ndmi)),
        'endmi_std': float(np.nanstd(ndmi)),
    }

def extract_natural_color(image: np.ndarray) -> np.ndarray:
    """
    Extract a natural color composite from the processed image.
    Uses:
      - Red   = B4 (channel 2)
      - Green = B3 (channel 1)
      - Blue  = B2 (channel 0)
    """
    return image[..., [2, 1, 0]]

def normalize_to_uint8(img: np.ndarray) -> np.ndarray:
    """
    Normalize a 2D or 3D image to [0, 255] as uint8 (done per channel).
    """
    if img.ndim == 2:
        min_val, max_val = img.min(), img.max()
        if max_val - min_val < 1e-6:
            return np.zeros_like(img, dtype=np.uint8)
        norm = (img - min_val) / (max_val - min_val)
        return (norm * 255).astype(np.uint8)
    elif img.ndim == 3:
        out = []
        for c in range(img.shape[-1]):
            channel = img[..., c]
            min_val, max_val = channel.min(), channel.max()
            if max_val - min_val < 1e-6:
                norm = np.zeros_like(channel)
            else:
                norm = (channel - min_val) / (max_val - min_val)
            out.append((norm * 255).astype(np.uint8))
        return np.stack(out, axis=-1)
    else:
        raise ValueError("Unsupported image dimension.")

def get_timm_features(pil_img: Image.Image, transform, model, device) -> np.ndarray:
    """
    Extract features from a PIL image using the given TIMM model.
    Returns a flattened feature vector.
    """
    input_tensor = transform(pil_img).unsqueeze(0).to(device)  # shape: (1, 3, H, W)
    with torch.no_grad():
        features = model(input_tensor)
    return features.cpu().numpy().flatten()

def compute_additional_band_stats(image: np.ndarray) -> dict:
    """
    Compute extra statistics (skewness and kurtosis) per band.
    These additional features can capture distribution characteristics.
    """
    stats = {}
    for ch in range(image.shape[-1]):
        stats[f'band{ch}_skew'] = float(skew(image[..., ch].ravel()))
        stats[f'band{ch}_kurtosis'] = float(kurtosis(image[..., ch].ravel()))
    return stats





# =============================================================================
# Function For Transforming V3
# =============================================================================




# =============================================================================
# Debug Function for Images
# =============================================================================
def debug_images():
    """
    Debug function to display one sample event's images.
    Displays:
      - Raw image (natural-color composite)
      - Processed image (after normalization and slope decoding)
      - Final image (8-bit RGB for viewing)
    Also prints out shapes.
    """
    BASE_PATH = './data'
    COMPOSITE_IMAGES_PATH = os.path.join(BASE_PATH, 'composite_images.npz')
    images = np.load(COMPOSITE_IMAGES_PATH)
    
    sample_event = list(images.keys())[0]
    raw_img = images[sample_event]
    
    print(f"[DEBUG] Sample event_id: {sample_event}")
    print("[DEBUG] Raw image shape:", raw_img.shape)
    
    if raw_img.ndim == 4:
        print("[DEBUG] Image is a time series. Using the first snapshot.")
        snapshot = raw_img[0]
        print("[DEBUG] Snapshot shape:", snapshot.shape)
    elif raw_img.ndim == 3:
        print("[DEBUG] Image is static.")
        snapshot = raw_img
    else:
        print("[DEBUG] Unexpected image dimensions:", raw_img.ndim)
        return

    if snapshot.shape[-1] >= 3:
        raw_rgb = snapshot[..., [2, 1, 0]]
        raw_rgb_norm = (raw_rgb - raw_rgb.min()) / (raw_rgb.max() - raw_rgb.min() + 1e-6)
    else:
        raw_rgb_norm = snapshot

    proc_img = preprocess_image(snapshot)
    print("[DEBUG] Processed image shape:", proc_img.shape)
    proc_rgb = extract_natural_color(proc_img)
    proc_rgb_norm = (proc_rgb - proc_rgb.min()) / (proc_rgb.max() - proc_rgb.min() + 1e-6)
    final_img = normalize_to_uint8(proc_rgb)
    print("[DEBUG] Final image shape (uint8):", final_img.shape)
    
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(raw_rgb_norm)
    plt.title("Raw Image (Natural Color Composite)")
    plt.axis("off")
    
    plt.subplot(1, 3, 2)
    plt.imshow(proc_rgb_norm)
    plt.title("Processed Image (Normalized)")
    plt.axis("off")
    
    plt.subplot(1, 3, 3)
    plt.imshow(final_img)
    plt.title("Final Image (8-bit RGB)")
    plt.axis("off")
    
    plt.tight_layout()
    plt.show()

# =============================================================================
# Autoencoder for TIMM Feature Reduction
# =============================================================================
class FeatureAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(FeatureAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, input_dim)
        )
    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return z, x_recon

def train_feature_autoencoder(features, latent_dim, epochs, lr, device):
    """
    Train an autoencoder on the TIMM features.
    Returns the encoded (latent) features.
    """
    input_dim = features.shape[1]
    model = FeatureAutoencoder(input_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        z, x_recon = model(features_tensor)
        loss = loss_fn(x_recon, features_tensor)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"[AUTOENCODER] Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")
    model.eval()
    with torch.no_grad():
        z, _ = model(features_tensor)
    return z.cpu().numpy()

# =============================================================================
# Attention Fusion Autoencoder for Multimodal Fusion
# =============================================================================
class AttentionFusionAE(nn.Module):
    def __init__(self, img_dim, tab_dim, fusion_dim):
        super(AttentionFusionAE, self).__init__()
        self.fc_img = nn.Linear(img_dim, fusion_dim)
        self.fc_tab = nn.Linear(tab_dim, fusion_dim)
        self.attn = nn.Linear(fusion_dim, 1)
        # Decoders for reconstruction (unsupervised training objective)
        self.decoder_img = nn.Linear(fusion_dim, img_dim)
        self.decoder_tab = nn.Linear(fusion_dim, tab_dim)
    def forward(self, img_feat, tab_feat):
        img_proj = self.fc_img(img_feat)
        tab_proj = self.fc_tab(tab_feat)
        fusion = torch.tanh(img_proj + tab_proj)
        weight = torch.sigmoid(self.attn(fusion))  # (N, 1)
        fused = weight * img_proj + (1 - weight) * tab_proj  # (N, fusion_dim)
        rec_img = self.decoder_img(fused)
        rec_tab = self.decoder_tab(fused)
        return fused, rec_img, rec_tab

def train_attention_fusion(img_feats, tab_feats, fusion_dim, epochs, lr, device):
    """
    Train an attention fusion autoencoder on image and tabular features.
    Returns the fused features.
    """
    img_dim = img_feats.shape[1]
    tab_dim = tab_feats.shape[1]
    model = AttentionFusionAE(img_dim, tab_dim, fusion_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    img_tensor = torch.tensor(img_feats, dtype=torch.float32).to(device)
    tab_tensor = torch.tensor(tab_feats, dtype=torch.float32).to(device)
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        fused, rec_img, rec_tab = model(img_tensor, tab_tensor)
        loss_img = loss_fn(rec_img, img_tensor)
        loss_tab = loss_fn(rec_tab, tab_tensor)
        loss = loss_img + loss_tab
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"[FUSION AE] Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")
    model.eval()
    with torch.no_grad():
        fused, _, _ = model(img_tensor, tab_tensor)
    return fused.cpu().numpy()


In [27]:
def run_pipelineV1(debug=False, img_dimension =224, MODEL_NAME = None):
    """
    Run the full feature-engineering pipeline.
    
    If debug=True, prints detailed information about:
      - Data shapes (raw, processed, features)
      - Model name and TIMM feature vector shape per event
      - PCA output shapes for each setting
      - Final dataframe shape and column names
      - Also displays image diagnostics via debug_images()
    
    Returns:
        df_features: A pandas DataFrame with one row per event and all engineered features.
    """
    BASE_PATH = work_dir #'./data'
    TRAIN_CSV = os.path.join(BASE_PATH, 'Train.csv')
    TEST_CSV = os.path.join(BASE_PATH, 'Test.csv')
    COMPOSITE_IMAGES_PATH = os.path.join(BASE_PATH, 'composite_images.npz')
    
    # ------------------------------
    # Load CSV files and extract event IDs
    # ------------------------------
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    train_df['event_id'] = train_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
    test_df['event_id'] = test_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
    train_event_ids = train_df['event_id'].unique().tolist()
    test_event_ids = test_df['event_id'].unique().tolist()
    all_event_ids = list(set(train_event_ids) | set(test_event_ids))
    if debug:
        print(f"[DEBUG] Total unique event ids: {len(all_event_ids)}")
    
    # ------------------------------
    # Load Composite Images
    # ------------------------------
    images = np.load(COMPOSITE_IMAGES_PATH)
    sample_key = list(images.keys())[0]
    sample_img = images[sample_key]
    if sample_img.ndim == 4:
        if debug:
            print("[DEBUG] Composite images have a time series dimension; averaging over time.")
    elif sample_img.ndim == 3:
        if debug:
            print("[DEBUG] Composite images are static (one frame per event).")
    else:
        print("[DEBUG] Unexpected image dimensions:", sample_img.ndim)
        return
    
    # ------------------------------
    # Set up device and TIMM model
    # ------------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    timm_model_name = MODEL_NAME
    timm_model = timm.create_model(timm_model_name, pretrained=True, num_classes=0)
    timm_model.to(device)
    timm_model.eval()
    if debug:
        print(f"[DEBUG] TIMM Model: {timm_model_name} (classifier removed)")
    
    transform = create_transform(
        input_size=img_dimension,
        is_training=False,
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
    
    # ------------------------------
    # Loop over events to compute features
    # ------------------------------
    features_dict = {}
    timm_features_list = []
    event_ids_list = []
    
    for event_id in tqdm(all_event_ids, desc="Processing events"):
        if event_id not in images:
            if debug:
                print(f"[DEBUG] Warning: event_id {event_id} not found in composite images. Skipping.")
            continue
        
        raw_img = images[event_id]
        proc_img = preprocess_image(raw_img)
        
        # Compute hand-engineered features.
        band_stats = compute_band_stats(proc_img)
        indices_stats = compute_indices(proc_img)
        
        # Prepare natural-color image for TIMM extraction.
        natural_img = extract_natural_color(proc_img)
        natural_img_uint8 = normalize_to_uint8(natural_img)
        pil_img = Image.fromarray(natural_img_uint8)
        
        # Extract TIMM features.
        timm_feat = get_timm_features(pil_img, transform, timm_model, device)
        # if debug:
        #     print(f"[DEBUG] Event {event_id}: processed image shape {proc_img.shape}, TIMM feature vector shape {timm_feat.shape}")
        
        feats = {}
        feats.update(band_stats)
        feats.update(indices_stats)
        feats['timm_feat_dim'] = len(timm_feat)
        features_dict[event_id] = feats
        timm_features_list.append(timm_feat)
        event_ids_list.append(event_id)
    
    df_features = pd.DataFrame.from_dict(features_dict, orient='index')
    df_features.index.name = 'event_id'
    
    timm_features_matrix = np.vstack(timm_features_list)
    if debug:
        print(f"[DEBUG] TIMM features matrix shape: {timm_features_matrix.shape}")
    
    # ------------------------------
    # PCA on TIMM features and append PCA columns
    # ------------------------------
    for n_components in [16, 32, 64, 128, 256]:
        pca = PCA(n_components=n_components, random_state = SEED_)
        pca_components = pca.fit_transform(timm_features_matrix)
        if debug:
            print(f"[DEBUG] PCA with {n_components} components, output shape: {pca_components.shape}")
        for i in range(n_components):
            col_name = f'pca{n_components}_{i}'
            df_features[col_name] = pd.Series(pca_components[:, i], index=event_ids_list)
    
    df_features.reset_index(inplace=True)
    if debug:
        print(f"[DEBUG] Final dataframe shape: {df_features.shape}")
        print(f"[DEBUG] Dataframe columns: {df_features.columns.tolist()}")
    
    # ------------------------------
    # If debugging, display sample images and diagnostics.
    # ------------------------------
    if debug:
        debug_images()
    
    return df_features

In [28]:

def run_pipelineV3(debug=False,
                 keep_original_image_features=True,
                 target_dim=100,
                 extra_image_features=False,
                 use_autoencoder=False,
                 autoencoder_epochs=50,
                 autoencoder_lr=1e-3,
                 use_attention_fusion=False,
                 fusion_dim=128,
                 fusion_epochs=50,
                 fusion_lr=1e-3,
                 img_dimension = 224,
                 MODEL_NAME = None
                 ):
    """
    Run the full feature-engineering pipeline with several toggles.
    
    Parameters:
      - debug: Print detailed debug information and display images.
      - keep_original_image_features: If True, keep full TIMM features; if False, reduce them.
      - target_dim: Target dimension for TIMM feature reduction (via PCA or autoencoder).
      - extra_image_features: Compute extra band statistics (skewness, kurtosis).
      - use_autoencoder: If True, use a trainable autoencoder to reduce TIMM features.
      - autoencoder_epochs, autoencoder_lr: Hyperparameters for the autoencoder.
      - use_attention_fusion: If True, fuse tabular (hand-engineered) and deep image features via an attention autoencoder.
      - fusion_dim, fusion_epochs, fusion_lr: Hyperparameters for the attention fusion autoencoder.
    
    Returns:
        df_features: A pandas DataFrame with one row per event containing all engineered features.
    """
    BASE_PATH = work_dir #'./data'
    TRAIN_CSV = os.path.join(BASE_PATH, 'Train.csv')
    TEST_CSV = os.path.join(BASE_PATH, 'Test.csv')
    COMPOSITE_IMAGES_PATH = os.path.join(BASE_PATH, 'composite_images.npz')
    
    # ------------------------------
    # Load CSVs and extract event IDs
    # ------------------------------
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    train_df['event_id'] = train_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
    test_df['event_id'] = test_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
    train_event_ids = train_df['event_id'].unique().tolist()
    test_event_ids = test_df['event_id'].unique().tolist()
    all_event_ids = list(set(train_event_ids) | set(test_event_ids))
    if debug:
        print(f"[DEBUG] Total unique event ids: {len(all_event_ids)}")
    
    # ------------------------------
    # Load Composite Images
    # ------------------------------
    images = np.load(COMPOSITE_IMAGES_PATH)
    sample_key = list(images.keys())[0]
    sample_img = images[sample_key]
    if sample_img.ndim == 4:
        if debug:
            print("[DEBUG] Composite images have a time series dimension; averaging over time.")
    elif sample_img.ndim == 3:
        if debug:
            print("[DEBUG] Composite images are static (one frame per event).")
    else:
        print("[DEBUG] Unexpected image dimensions:", sample_img.ndim)
        return
    
    # ------------------------------
    # Set up device and TIMM model
    # ------------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    timm_model_name = MODEL_NAME # "resnet50"
    timm_model = timm.create_model(timm_model_name, pretrained=True, num_classes=0)
    timm_model.to(device)
    timm_model.eval()
    if debug:
        print(f"[DEBUG] TIMM Model: {timm_model_name} (classifier removed)")
    
    transform = create_transform(
        input_size=img_dimension,
        is_training=False,
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
    
    # ------------------------------
    # Loop over events to compute features
    # ------------------------------
    features_dict = {}
    timm_features_list = []
    event_ids_list = []
    
    for event_id in tqdm(all_event_ids, desc="Processing events"):
        if event_id not in images:
            if debug:
                print(f"[DEBUG] Warning: event_id {event_id} not found in composite images. Skipping.")
            continue
        
        raw_img = images[event_id]
        proc_img = preprocess_image(raw_img)
        
        # Compute basic hand-engineered features.
        feats = {}
        feats.update(compute_band_stats(proc_img))
        feats.update(compute_indices(proc_img))
        if extra_image_features:
            feats.update(compute_additional_band_stats(proc_img))
        
        # Prepare natural-color image for TIMM extraction.
        natural_img = extract_natural_color(proc_img)
        natural_img_uint8 = normalize_to_uint8(natural_img)
        pil_img = Image.fromarray(natural_img_uint8)
        
        # Extract TIMM features.
        timm_feat = get_timm_features(pil_img, transform, timm_model, device)
        if debug:
            print(f"[DEBUG] Event {event_id}: processed image shape {proc_img.shape}, TIMM feature vector shape {timm_feat.shape}")
        
        feats['timm_feat_dim'] = len(timm_feat)
        features_dict[event_id] = feats
        timm_features_list.append(timm_feat)
        event_ids_list.append(event_id)
    
    # Create dataframe from hand-engineered features.
    df_features = pd.DataFrame.from_dict(features_dict, orient='index')
    df_features.index.name = 'event_id'
    
    timm_features_matrix = np.vstack(timm_features_list)
    if debug:
        print(f"[DEBUG] TIMM features matrix shape: {timm_features_matrix.shape}")
    
    # ------------------------------
    # Process TIMM features: Either keep, reduce via PCA, or via Autoencoder.
    # ------------------------------
    if keep_original_image_features:
        if debug:
            print("[DEBUG] Keeping original TIMM features.")
        for i in range(timm_features_matrix.shape[1]):
            col_name = f'timm_orig_{i}'
            df_features[col_name] = pd.Series(timm_features_matrix[:, i], index=df_features.index)
        deep_features = timm_features_matrix  # for later fusion if needed
    else:
        if use_autoencoder:
            if debug:
                print(f"[DEBUG] Reducing TIMM features using Autoencoder to {target_dim} dims over {autoencoder_epochs} epochs.")
            reduced = train_feature_autoencoder(timm_features_matrix, target_dim, autoencoder_epochs, autoencoder_lr, device)
            for i in range(reduced.shape[1]):
                col_name = f'timm_autoencoder_{i}'
                df_features[col_name] = pd.Series(reduced[:, i], index=df_features.index)
            deep_features = reduced
        else:
            if debug:
                print(f"[DEBUG] Reducing TIMM features using PCA to {target_dim} dims.")
            pca = PCA(n_components=target_dim)
            reduced = pca.fit_transform(timm_features_matrix)
            if debug:
                print(f"[DEBUG] PCA reduced features shape: {reduced.shape}")
            for i in range(reduced.shape[1]):
                col_name = f'timm_pca_{i}'
                df_features[col_name] = pd.Series(reduced[:, i], index=df_features.index)
            deep_features = reduced
    
    # ------------------------------
    # Optional Attention Fusion between deep and tabular features.
    # ------------------------------
    if use_attention_fusion:
        if debug:
            print(f"[DEBUG] Applying Attention Fusion with fusion dimension: {fusion_dim} over {fusion_epochs} epochs.")
        all_cols = df_features.columns.tolist()
        deep_feature_cols = [col for col in all_cols if col.startswith('timm_')]
        tabular_feature_cols = [col for col in all_cols if col not in deep_feature_cols]
        img_feats = df_features[deep_feature_cols].values
        tab_feats = df_features[tabular_feature_cols].values
        fused_features = train_attention_fusion(img_feats, tab_feats, fusion_dim, fusion_epochs, fusion_lr, device)
        for i in range(fused_features.shape[1]):
            col_name = f'fusion_{i}'
            df_features[col_name] = pd.Series(fused_features[:, i], index=df_features.index)
    
    # Optionally reset index to include event_id as a column.
    df_features.reset_index(inplace=True)
    if debug:
        print(f"[DEBUG] Final dataframe shape: {df_features.shape}")
        print(f"[DEBUG] Dataframe columns (first 10): {df_features.columns.tolist()[:10]}")
    
    if debug:
        debug_images()
    
    return df_features

## Execute & Save Features Extracted

In [29]:
for (model_, img_) in v1_models:
    df = run_pipelineV1(debug=False, img_dimension = img_, MODEL_NAME = model_ )
    df.to_parquet(f'{FEAT_PATH_}/Processed_V1_{model_}_S{SEED_}.parquet', engine='pyarrow')

for (model_, img_) in v3_models:
    df = run_pipelineV3(debug=False,
                keep_original_image_features=False,  # reduce deep features
                target_dim=100,
                extra_image_features=True,
                use_autoencoder=True,     # use autoencoder reduction instead of PCA
                autoencoder_epochs=110,
                autoencoder_lr=1e-3,
                use_attention_fusion=True,  # enable attention fusion of deep and tabular features
                fusion_dim=128,
                fusion_epochs=50,
                fusion_lr=1e-3, img_dimension = img_, MODEL_NAME = model_)
    df.to_parquet(f'{FEAT_PATH_}/Processed_V3_{model_}_S{SEED_}.parquet', engine='pyarrow')


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

Processing events: 100%|██████████| 898/898 [00:14<00:00, 59.98it/s]


model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

Processing events: 100%|██████████| 898/898 [00:08<00:00, 110.81it/s]


pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

...

[AUTOENCODER] Epoch 0/110, Loss: 0.0684
[AUTOENCODER] Epoch 10/110, Loss: 0.0438
[AUTOENCODER] Epoch 20/110, Loss: 0.0334
[AUTOENCODER] Epoch 30/110, Loss: 0.0265
[AUTOENCODER] Epoch 40/110, Loss: 0.0219
[AUTOENCODER] Epoch 50/110, Loss: 0.0184
[AUTOENCODER] Epoch 60/110, Loss: 0.0162
[AUTOENCODER] Epoch 70/110, Loss: 0.0136
[AUTOENCODER] Epoch 80/110, Loss: 0.0119
[AUTOENCODER] Epoch 90/110, Loss: 0.0105
[AUTOENCODER] Epoch 100/110, Loss: 0.0093
[FUSION AE] Epoch 0/50, Loss: 79554.2734
[FUSION AE] Epoch 10/50, Loss: 56161.3906
[FUSION AE] Epoch 20/50, Loss: 26546.5000
[FUSION AE] Epoch 30/50, Loss: 14188.9590
[FUSION AE] Epoch 40/50, Loss: 9252.1924


Processing events: 100%|██████████| 898/898 [00:13<00:00, 68.18it/s]


[AUTOENCODER] Epoch 0/110, Loss: 0.0655
[AUTOENCODER] Epoch 10/110, Loss: 0.0273
[AUTOENCODER] Epoch 20/110, Loss: 0.0223
[AUTOENCODER] Epoch 30/110, Loss: 0.0195
[AUTOENCODER] Epoch 40/110, Loss: 0.0168
[AUTOENCODER] Epoch 50/110, Loss: 0.0147
[AUTOENCODER] Epoch 60/110, Loss: 0.0127
[AUTOENCODER] Epoch 70/110, Loss: 0.0112
[AUTOENCODER] Epoch 80/110, Loss: 0.0098
[AUTOENCODER] Epoch 90/110, Loss: 0.0088
[AUTOENCODER] Epoch 100/110, Loss: 0.0080
[FUSION AE] Epoch 0/50, Loss: 58070.3438
[FUSION AE] Epoch 10/50, Loss: 41333.2930
[FUSION AE] Epoch 20/50, Loss: 11938.2627
[FUSION AE] Epoch 30/50, Loss: 2363.0186
[FUSION AE] Epoch 40/50, Loss: 2543.5928


Processing events: 100%|██████████| 898/898 [00:14<00:00, 60.86it/s]


[AUTOENCODER] Epoch 0/110, Loss: 0.8750
[AUTOENCODER] Epoch 10/110, Loss: 0.2044
[AUTOENCODER] Epoch 20/110, Loss: 0.1511
[AUTOENCODER] Epoch 30/110, Loss: 0.1294
[AUTOENCODER] Epoch 40/110, Loss: 0.1154
[AUTOENCODER] Epoch 50/110, Loss: 0.1037
[AUTOENCODER] Epoch 60/110, Loss: 0.0935
[AUTOENCODER] Epoch 70/110, Loss: 0.0838
[AUTOENCODER] Epoch 80/110, Loss: 0.0752
[AUTOENCODER] Epoch 90/110, Loss: 0.0684
[AUTOENCODER] Epoch 100/110, Loss: 0.0630
[FUSION AE] Epoch 0/50, Loss: 64260.3672
[FUSION AE] Epoch 10/50, Loss: 37654.0078
[FUSION AE] Epoch 20/50, Loss: 11885.1826
[FUSION AE] Epoch 30/50, Loss: 4690.3169
[FUSION AE] Epoch 40/50, Loss: 4127.8643


## Merge Processed Features to Single DF

In [36]:

def merge_parquet_files(directory='./data/Features_build', zero_only=True):
    """
    Reads all parquet files in `directory` whose filenames start with "Processed_V",
    renames their columns (except 'event_id') by appending a file-specific suffix,
    and merges them on 'event_id'.

    If zero_only is True, the function filters out PCA columns whose original 
    component (before adding the file suffix) is not "0". For PCA columns with a
    component of "0", it removes the trailing "_0" and file suffix.

    Parameters:
      - directory (str): Directory to search for parquet files.
      - zero_only (bool): If True, only PCA columns ending with component "0" are kept.
    
    Returns:
      - merged_df (pd.DataFrame): The merged (and possibly filtered) DataFrame.
    """
    # Find all files matching the pattern (e.g., ProcessV1_*.parquet)
    file_pattern = os.path.join(directory, 'Processed_V1_*.parquet')
    files = glob.glob(file_pattern)
    if not files:
        print("No files found matching the pattern.")
        return None

    merged_df = None

    for file_path in files:
        # Extract file-specific suffix:
        # Example: "ProcessV1_A.parquet" -> suffix "A"
        base_name = os.path.basename(file_path)
        suffix = base_name[len("Processed_V1_"):-len(".parquet")]

        # Read the parquet file.
        df = pd.read_parquet(file_path)

        # Rename all columns (except 'event_id') by appending the file-specific suffix.
        df = df.rename(columns={
            col: col if col == 'event_id' else f"{col}_{suffix}"
            for col in df.columns
        })

        # Merge sequentially on 'event_id'
        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='event_id', how='outer')

    # --- Filtering PCA columns if zero_only is True ---
    if zero_only:
        drop_cols = []
        rename_map = {}

        # Process each column in the merged dataframe.
        for col in merged_df.columns:
            # Check in a case-insensitive manner if the column is a PCA column.
            if col.lower().startswith("pca"):
                # We assume the structure is: <base>_<component>_<file_suffix>
                # For example:
                #   Original column: "pca256_251" in file, becomes "pca256_251_resnet34" after renaming.
                #   We want to keep only those where the component is "0", e.g. "pca256_0_resnet34"
                #
                # First, remove the file-specific suffix by splitting on the last underscore.
                if '_' not in col:
                    continue  # Skip unexpected format
                try:
                    original_part, file_suffix = col.rsplit('_', 1)
                except ValueError:
                    continue

                # Next, split original_part on its last underscore to get the component.
                if '_' not in original_part:
                    continue
                try:
                    base_feature, component = original_part.rsplit('_', 1)
                except ValueError:
                    continue

                # If the component is not "0", mark this column for dropping.
                if component != "0":
                    drop_cols.append(col)
                else:
                    # For PCA columns with component "0", rename the column to the base feature.
                    # For example: "pca256_0_resnet34" becomes "pca256"
                    rename_map[col] = f'{base_feature}_CL'

        # Drop all PCA columns that don't have component "0"
        if drop_cols:
            merged_df.drop(columns=drop_cols, inplace=True)
        # Rename the kept PCA columns
        if rename_map:
            merged_df.rename(columns=rename_map, inplace=True)

    return merged_df


def merge_processv3_files(directory='./data/Features_build'):
    """
    Reads all parquet files in `directory` whose filenames start with "Processed_V3",
    renames their columns (except 'event_id') by appending a file-specific suffix,
    and merges them on 'event_id' (outer join).

    Unlike the ProcessV1 merge, this function does not apply any PCA filtering, so all
    columns from the ProcessV3 files are retained.

    Parameters:
      directory (str): Directory to search for parquet files.
    
    Returns:
      merged_df (pd.DataFrame): The merged DataFrame containing all columns.
    """
    # Find all ProcessV3 files.
    file_pattern = os.path.join(directory, 'Processed_V3_*.parquet')
    files = glob.glob(file_pattern)
    if not files:
        print("No ProcessV3 files found matching the pattern.")
        return None

    merged_df = None

    for file_path in files:
        # Extract the file-specific suffix.
        # Example: "ProcessV3_modelA.parquet" yields suffix "modelA"
        base_name = os.path.basename(file_path)
        suffix = base_name[len("Processed_V3_"):-len(".parquet")]

        # Read the parquet file.
        df = pd.read_parquet(file_path)

        # Rename all columns (except 'event_id') by appending the file-specific suffix.
        df = df.rename(columns={
            col: col if col == 'event_id' else f"{col}_{suffix}"
            for col in df.columns
        })

        # Merge sequentially on 'event_id' using an outer join.
        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='event_id', how='outer')

    return merged_df


def remove_duplicate_merge_columns(df, suffix_x='_x', suffix_y='_y'):
    """
    Given a DataFrame resulting from a merge that might contain duplicate columns
    with suffixes (e.g. _x and _y), this function checks for matching pairs.
    If the values in the pair are identical, it drops the _y column and renames the
    _x column to remove the suffix.

    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        suffix_x (str): Suffix for columns from the left DataFrame (default '_x').
        suffix_y (str): Suffix for columns from the right DataFrame (default '_y').

    Returns:
        pd.DataFrame: The processed DataFrame with duplicate columns removed.
    """
    # We make a list of column names to iterate over because we may change the DataFrame during iteration.
    for col in list(df.columns):
        if col.endswith(suffix_x):
            # Get the base name by removing the _x suffix.
            base_name = col[:-len(suffix_x)]
            col_y = base_name + suffix_y
            if col_y in df.columns:
                # Check if the two columns are identical
                if df[col].equals(df[col_y]):
                    # Drop the _y column
                    df.drop(columns=[col_y], inplace=True)
                    # Rename the _x column to remove the suffix so that it has the base name.
                    df.rename(columns={col: base_name}, inplace=True)
    return df



def drop_or_rename_duplicate_columns_all(df, rename_suffix='_dup'):
    """
    Processes duplicate columns in a DataFrame.
    
    For each set of duplicate columns (i.e. columns with the same name):
      - The first occurrence is kept unchanged.
      - For any subsequent duplicate column:
          - If its values are identical to the first occurrence, it is dropped.
          - Otherwise, it is renamed by appending a suffix (default: '_dup') plus an incrementing counter.
    
    Parameters:
      df (pd.DataFrame): The input DataFrame that may contain duplicate column names.
      rename_suffix (str): Suffix used when renaming a duplicate column whose values differ.
    
    Returns:
      pd.DataFrame: A new DataFrame with duplicate columns removed or renamed so that all columns are unique.
    """
    # We'll build new column names for the columns we keep.
    new_columns = [None] * len(df.columns)
    # Keep track of which column indices to keep.
    indices_to_keep = []
    # Dictionary to record, for each column name, the first occurrence's Series and a count of how many we've seen.
    seen = {}
    
    for i, col in enumerate(df.columns):
        current_series = df.iloc[:, i]
        if col not in seen:
            # First occurrence: keep it with the original name.
            seen[col] = {
                'first_series': current_series,
                'count': 1
            }
            new_columns[i] = col
            indices_to_keep.append(i)
        else:
            # Already seen; compare with the first occurrence.
            first_series = seen[col]['first_series']
            if current_series.equals(first_series):
                # Values are identical, so we want to drop this column.
                new_columns[i] = None
                # (Do not add its index to indices_to_keep.)
            else:
                # Values differ: rename this duplicate column.
                seen[col]['count'] += 1
                new_name = f"{col}{rename_suffix}{seen[col]['count']}"
                new_columns[i] = new_name
                indices_to_keep.append(i)
    
    # Build the new DataFrame using only the kept columns.
    final_columns = [new_columns[i] for i in indices_to_keep]
    new_df = df.iloc[:, indices_to_keep].copy()
    new_df.columns = final_columns
    return new_df

In [37]:
print(FEAT_PATH_)
merged_v1 = merge_parquet_files(directory=FEAT_PATH_, zero_only=True)
merged_v3 = merge_processv3_files(directory=FEAT_PATH_)

./data/Features_build


In [38]:
images_ = pd.merge(left=merged_v1, right=merged_v3, on = 'event_id')
images_ = remove_duplicate_merge_columns(images_)
images_ = drop_or_rename_duplicate_columns_all(images_, rename_suffix='_dup')


In [40]:
print(images_.shape)

(898, 806)


# 2. Train on Images Features

## Prepare Data & Merge to Features

In [41]:
def split_and_sum(df, ):
    # Create a new column by splitting 'event_id' at '_X_' and taking the first part
    df['base_event_id'] = df['event_id'].str.split('_X_').str[0]

    # Helper function: Count the minimum number of incidents (sorted in descending order)
    # required to reach a cumulative precipitation sum >= 100.
    def count_to_reach_100(x):
        # Sort precipitation values in descending order
        sorted_vals = x.sort_values(ascending=False)
        # Compute the cumulative sum
        cum_sum = sorted_vals.cumsum()
        # Check where cumulative sum reaches or exceeds 100
        cond = cum_sum >= 100
        if cond.any():
            # np.argmax returns the first index where condition is True.
            return int(np.argmax(cond.to_numpy()) + 1)
        else:
            # If 100 is never reached, return the total number of incidents.
            return len(x)

    
    # Group by the new base_event_id and aggregate the columns
    aggregated_df = df.groupby('base_event_id', as_index=False).agg(
        label = ('label', 'sum'),
        precipitation_min = ('precipitation', 'min'),
        precipitation_max = ('precipitation', 'max'),
        precipitation_mean = ('precipitation', 'mean'),
        precipitation_mode = ('precipitation', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
        precipitation_sum=('precipitation', 'sum'),
        precipitation_std=('precipitation', 'std'),
        precipitation_count_0=('precipitation', lambda x: (x == 0).sum()),
        precipitation_count_less_5=('precipitation', lambda x: (x < 5).sum()),
        precipitation_count_between_5_and_10=('precipitation', lambda x: ((x >= 5) & (x < 10)).sum()),
        precipitation_count_to_reach_100=('precipitation', count_to_reach_100)

    )

    
    return aggregated_df


In [42]:
train_ = pd.read_csv(train_file_path)
test_ = pd.read_csv(test_file_path)
train_.head()

,event_id,precipitation,label
0,id_spictby0jfsb_X_0,0.000000,0
1,id_spictby0jfsb_X_1,0.095438,0
2,id_spictby0jfsb_X_2,1.949560,0
3,id_spictby0jfsb_X_3,3.232160,0
4,id_spictby0jfsb_X_4,0.000000,0


In [43]:
test_['base_event_id'] = test_['event_id'].str.split('_X_').str[0]
test_['label'] = 0


train = split_and_sum(train_)
train['Type'] = 'Train'

test = split_and_sum(test_)
test['Type'] = 'Test'


print(train.shape)
train.head()

(674, 13)


,base_event_id,label,precipitation_min,precipitation_max,precipitation_mean,precipitation_mode,precipitation_sum,precipitation_std,precipitation_count_0,precipitation_count_less_5,precipitation_count_between_5_and_10,precipitation_count_to_reach_100,Type
0,id_05v6zjuaf300,1,0.0,54.3969,2.181141,0.0,1592.232796,6.536875,581,637,32,2,Train
1,id_06zma02zeea7,0,0.0,51.9134,1.198987,0.0,875.260653,4.075114,602,666,38,3,Train
2,id_08w2po0cz63y,0,0.0,10.7278,0.266291,0.0,194.392672,0.911627,615,726,3,24,Train
3,id_092vetuky9ku,0,0.0,28.8956,0.276754,0.0,202.030696,1.613950,668,720,6,9,Train
4,id_0987b1h04r48,1,0.0,86.1455,1.882442,0.0,1374.182907,6.736390,577,649,42,2,Train


In [44]:
full_data = pd.concat([train, test], ignore_index= True)
print(full_data.shape)
full_data

(898, 13)


,base_event_id,label,precipitation_min,precipitation_max,precipitation_mean,precipitation_mode,precipitation_sum,precipitation_std,precipitation_count_0,precipitation_count_less_5,precipitation_count_between_5_and_10,precipitation_count_to_reach_100,Type
0,id_05v6zjuaf300,1,0.0,54.3969,2.181141,0.0,1592.232796,6.536875,581,637,32,2,Train
1,id_06zma02zeea7,0,0.0,51.9134,1.198987,0.0,875.260653,4.075114,602,666,38,3,Train
2,id_08w2po0cz63y,0,0.0,10.7278,0.266291,0.0,194.392672,0.911627,615,726,3,24,Train
3,id_092vetuky9ku,0,0.0,28.8956,0.276754,0.0,202.030696,1.613950,668,720,6,9,Train
4,id_0987b1h04r48,1,0.0,86.1455,1.882442,0.0,1374.182907,6.736390,577,649,42,2,Train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,id_zgckftx7lo64,0,0.0,31.4639,1.786603,0.0,1304.220480,4.800234,566,640,41,4,Test
894,id_zibfeb9w8fl5,0,0.0,47.6422,2.338661,0.0,1707.222526,5.973153,538,613,52,3,Test
895,id_zk50aeed9xce,0,0.0,32.6604,1.363975,0.0,995.701972,4.401754,591,665,27,4,Test
896,id_zl678jv8h8u7,0,0.0,52.2699,1.549485,0.0,1131.124059,5.076125,582,660,28,3,Test


In [45]:
df = pd.merge(left = images_, right = full_data, left_on = 'event_id', right_on = 'base_event_id', how = 'left')
df.head()

,event_id,band0_mean_efficientnet_b3.ra2_in1k_S42,band0_std_efficientnet_b3.ra2_in1k_S42,band1_mean_efficientnet_b3.ra2_in1k_S42,band1_std_efficientnet_b3.ra2_in1k_S42,band2_mean_efficientnet_b3.ra2_in1k_S42,band2_std_efficientnet_b3.ra2_in1k_S42,band3_mean_efficientnet_b3.ra2_in1k_S42,band3_std_efficientnet_b3.ra2_in1k_S42,band4_mean_efficientnet_b3.ra2_in1k_S42,...,precipitation_max,precipitation_mean,precipitation_mode,precipitation_sum,precipitation_std,precipitation_count_0,precipitation_count_less_5,precipitation_count_between_5_and_10,precipitation_count_to_reach_100,Type
0,id_05v6zjuaf300,-0.351159,0.387995,-0.488726,0.448289,-0.457963,0.590306,1.270173,0.873391,1.149284,...,54.3969,2.181141,0.0,1592.232796,6.536875,581,637,32,2,Train
1,id_066zz28m11mr,-0.267915,0.097005,-0.132496,0.174218,0.570542,0.322990,1.115806,0.377975,2.640203,...,35.8728,0.385397,0.0,281.339884,1.884940,644,709,19,8,Test
2,id_06zma02zeea7,0.126439,0.208946,0.289442,0.312349,1.065926,0.582471,2.389661,0.619813,5.054065,...,51.9134,1.198987,0.0,875.260653,4.075114,602,666,38,3,Train
3,id_073l04ir88sn,0.969935,0.433456,0.774585,0.467205,1.052091,0.595688,1.732535,0.795263,2.823037,...,162.3970,2.116429,0.0,1544.993471,9.632064,567,654,36,1,Test
4,id_08w2po0cz63y,-0.058516,0.095812,-0.026293,0.144664,0.402103,0.194872,0.659779,0.212870,1.796405,...,10.7278,0.266291,0.0,194.392672,0.911627,615,726,3,24,Train


## Run Images Training - Separated

### TRAIN - R4 Version

In [55]:
params_0 =  {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'n_estimators': 5_000,
        'learning_rate': 0.025, 
        'feature_fraction': 0.85, 
        'bagging_fraction': 0.92, 
        'bagging_freq': 40, 
        'num_leaves': 99,
        'verbose': -1,
        'seed': 42,
    }

params_1 =  {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'n_estimators': 5_000,
    'learning_rate': 0.00820777905418306, 
    'feature_fraction': 0.8017433735476059, 
    'bagging_fraction': 0.9379166782699045, 
    'bagging_freq': 43, 
    'num_leaves': 95,
    'verbose': -1,
    'seed': 42,
}

params_2 =  {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'n_estimators': 5_000,
    'learning_rate': 0.033893435894644865, 
    'feature_fraction': 0.857238873524995, 
    'bagging_fraction': 0.7055416618687687, 
    'bagging_freq': 32, 
    'num_leaves': 30,
    'verbose': -1,
    'seed': 42,
}
params_3 =   {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    

    'learning_rate': 0.02872672034985086, 
    'feature_fraction': 0.8573811600751811, 
    'bagging_fraction': 0.7051969782608369, 
    'bagging_freq': 23, 
    'num_leaves': 62, 
    'n_estimators': 5796, 
    'max_depth': 45,        

    'verbose': -1,
    'seed': 42,
}





In [52]:
def train_lgbm_with_cv(df, params):
    # --- 1. Split the full DataFrame into train and test using the 'Type' column ---
    # Assuming 'Type' column has values like 'train' and 'test'
    train_df = df[df['Type'] == 'Train'].copy()
    test_df = df[df['Type'] == 'Test'].copy()
    
    # Save event_id for train and test to attach later in OOF and test predictions
    train_ids = train_df[['event_id']].copy()
    test_ids  = test_df[['event_id']].copy()
    
    # Drop columns not used as features in training
    # We drop 'event_id', 'base_event_id', and 'Type'
    drop_cols = ['event_id', 'base_event_id', 'Type']
    train_df = train_df.drop(columns=drop_cols)
    test_df  = test_df.drop(columns=drop_cols)
    
    # Separate features and target for train data
    X = train_df.drop(columns=['label'])
    y = train_df['label']
    
    # For test, if label is present, drop it (typically test is unlabeled)
    X_test = test_df.drop(columns=['label'], errors='ignore')
    
    # Show label distribution to verify stratification will work
    print("Label distribution in train set:\n", y.value_counts())
    
    # Create 5 stratified folds on the train set
    skf = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)
    
    # Initialize arrays for out-of-fold predictions and test predictions
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    
    
    # 5-Fold Cross Validation
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        print(f"--- Fold {fold+1} ---")
        # Ensure we have non-empty splits
        if len(train_idx) == 0 or len(valid_idx) == 0:
            print(f"Skipping fold {fold+1} due to insufficient data.")
            continue
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # Prepare LightGBM datasets
        lgb_train = lgb.Dataset(X_train, label=y_train)
        lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

        
        # Train the model with early stopping
        model = lgb.train(
            params,
            lgb_train,
            num_boost_round=3000,
            valid_sets=[lgb_train, lgb_valid],
            valid_names=['train', 'valid'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=250, verbose=False),
            ]
        )
        
        # Predict on validation fold
        oof_preds[valid_idx] = model.predict(X_valid, num_iteration=model.best_iteration)
        
        # Predict on test set and average over folds
        test_preds += model.predict(X_test, num_iteration=model.best_iteration) / skf.n_splits

    # Evaluate model performance on training data using OOF predictions
    oof_logloss = log_loss(y, oof_preds)
    print("\nOOF Binary Log Loss:", oof_logloss)
    
    # Convert probabilities to binary predictions (using threshold 0.5)
    oof_binary = (oof_preds >= 0.5).astype(int)
    conf_mat = confusion_matrix(y, oof_binary)
    print("\nConfusion Matrix (OOF predictions):")
    print(conf_mat)
    
    # Create an OOF DataFrame that includes the original event_id, true label, and predictions
    oof_df = train_ids.copy()
    oof_df['label'] = y.values
    oof_df['oof_pred'] = oof_preds
    oof_df['predicted_label'] = oof_binary

    test_ids['preds'] =  test_preds
    test_ids['predicted_label'] = (test_preds >= 0.5).astype(int)

    
    return oof_df, test_preds, test_ids

In [67]:
oof_preds, test_preds, test_ids = train_lgbm_with_cv(df, params_1)

Label distribution in train set:
 label
0    356
1    318
Name: count, dtype: int64
--- Fold 1 ---
--- Fold 2 ---
--- Fold 3 ---
--- Fold 4 ---
--- Fold 5 ---
--- Fold 6 ---
--- Fold 7 ---
--- Fold 8 ---
--- Fold 9 ---
--- Fold 10 ---
--- Fold 11 ---
--- Fold 12 ---
--- Fold 13 ---
--- Fold 14 ---
--- Fold 15 ---

OOF Binary Log Loss: 0.23559340295319595

Confusion Matrix (OOF predictions):
[[329  27]
 [ 38 280]]


In [62]:
test_ids.to_csv(f'IDSx_TEST_FromImages_S{SEED_}_PR1.csv',index=False)
oof_preds.to_csv(f'IDSx_OOF_FromImages_S{SEED_}_PR1.csv',index=False)

In [ ]:
'''
==> 0
OOF Binary Log Loss: 0.23790857289877537 - x67
Confusion Matrix (OOF predictions):
[[330  26]
[ 41 277]]


==> 1
OOF Binary Log Loss: 0.23559340295319595 - x65
Confusion Matrix (OOF predictions):
[[329  27]
[ 38 280]]


==> 2
OOF Binary Log Loss: 0.23777142855402644 - x65
Confusion Matrix (OOF predictions):
[[332  24]
[ 41 277]]


==> 3
OOF Binary Log Loss: 0.23596330482794922 - x65
Confusion Matrix (OOF predictions):
[[329  27]
[ 38 280]]


'''

In [ ]:
## The END of Process & Usage of Images ## 

# 3. CSV Files & LAGS

In [ ]:
## The Start of CSV Files - LAGS ## 

In [63]:
data = pd.read_csv(train_file_path)
data_test = pd.read_csv(test_file_path)


data['event_id'] = data['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
data['event_idx'] = data.groupby('event_id', sort=False).ngroup()
data_test['event_id'] = data_test['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
data_test['event_idx'] = data_test.groupby('event_id', sort=False).ngroup()

data['event_t'] = data.groupby('event_id').cumcount()
data_test['event_t'] = data_test.groupby('event_id').cumcount()

In [64]:

def process_precipitation_lag_mean(df, lag_range=(1, 365)):
    """
    Processes a DataFrame by grouping by 'event_id' and creating lag features for the 'precipitation' column.
    Then, it calculates the mean of these lagged values.

    Args:
        df (pd.DataFrame): The input DataFrame containing at least 'event_id' and 'precipitation' columns.
        lag_range (tuple): A tuple specifying the range of lags to create (start, end).

    Returns:
        pd.DataFrame: A DataFrame with calculated lag mean columns.
    """
    
    # Check required columns exist
    if 'event_id' not in df.columns or 'precipitation' not in df.columns:
        raise ValueError("The DataFrame must contain 'event_id' and 'precipitation' columns.")

    # Extract the start and end of the lag range
    lag_start, lag_end = lag_range

    # Group by 'event_id' and process each group
    result = []
    for event_id, group in tqdm(df.groupby('event_id')):
        group = group.sort_index()  # Sort by index (assuming time series data)

        # Create lag features for the specified range
        for lag in range(lag_start, lag_end + 1):
            group[f'precipitation_lag_{lag}'] = group['precipitation'].shift(lag)

        # Calculate the mean of the lagged values
        lag_columns = [f'precipitation_lag_{lag}' for lag in range(lag_start, lag_end + 1)]
        group['precipitation_lag_mean'] = group[lag_columns].mean(axis=1)

        result.append(group)

    # Concatenate all groups back into a single DataFrame
    result_df = pd.concat(result)
    return result_df

In [66]:
prefix = 'V9'
data_          = process_precipitation_lag_mean(data, lag_range=(-365, 730))
data_test_     = process_precipitation_lag_mean(data_test, lag_range=(-365, 730))
data_.to_parquet(f"{work_dir}/Simple_Train{prefix}_RAW.parquet")
data_test_.to_parquet(f"{work_dir}/Simple_Test{prefix}_RAW.parquet")

100%|██████████| 224/224 [01:06<00:00,  3.34it/s]


# PART 2  -------------------------------

In [ ]:
import pandas as pd
import numpy as np
import os

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import log_loss
import lightgbm as lgb


import warnings
warnings.filterwarnings("ignore") 

In [ ]:
sin_version = 2

enable_tabular_predictions = True
tabular_targeted = '_S42_PR1'


data_dir        = 'data'


train_path_     =  "./data/Train.csv"
test_path_      =  './data/Test.csv'

SEED_ = 42

#image_feat_eng_path = "./data/Satellite_Features_ENG.csv"
#table_expanded_path = "./data/Expanded_PRECP_Features.csv"

#building_block_mode = 'single'
#building_block_path  = "./data/building_block_256.csv"

In [ ]:
def calculate_sin_cos_cycles(df, column_name, ver_ = 2):
    """
    Calculate sine and cosine values for different cycles (24-hour, 7-day, 30-day, 365-day, and normalized) 
    for a given column in a DataFrame.
    
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame")
    
    # Calculate sine and cosine for different cycles
    df['sin_24hr'] = np.sin(2 * np.pi * df[column_name] / 24)
    df['cos_24hr'] = np.cos(2 * np.pi * df[column_name] / 24)

    df['sin_7day'] = np.sin(2 * np.pi * df[column_name] / (7 * 24))
    df['cos_7day'] = np.cos(2 * np.pi * df[column_name] / (7 * 24))

    df['sin_14day'] = np.sin(2 * np.pi * df[column_name] / (14 * 24))
    df['cos_14day'] = np.cos(2 * np.pi * df[column_name] / (14 * 24))

    df['sin_21day'] = np.sin(2 * np.pi * df[column_name] / (21 * 24))
    df['cos_21day'] = np.cos(2 * np.pi * df[column_name] / (21 * 24))

    df['sin_30day'] = np.sin(2 * np.pi * df[column_name] / (30 * 24))
    df['cos_30day'] = np.cos(2 * np.pi * df[column_name] / (30 * 24))

    df['sin_60day'] = np.sin(2 * np.pi * df[column_name] / (60 * 24))
    df['cos_60day'] = np.cos(2 * np.pi * df[column_name] / (60 * 24))

    df['sin_365day'] = np.sin(2 * np.pi * df[column_name] / (365 * 24))
    df['cos_365day'] = np.cos(2 * np.pi * df[column_name] / (365 * 24))

    if ver_ in [2]:

        df['sin_2day'] = np.sin(2 * np.pi * df[column_name] / (2 * 24))
        df['cos_2day'] = np.cos(2 * np.pi * df[column_name] / (2 * 24))

        df['sin_3day'] = np.sin(2 * np.pi * df[column_name] / (3 * 24))
        df['cos_3day'] = np.cos(2 * np.pi * df[column_name] / (3 * 24))

        df['sin_4day'] = np.sin(2 * np.pi * df[column_name] / (4 * 24))
        df['cos_4day'] = np.cos(2 * np.pi * df[column_name] / (4 * 24))

        df['sin_5day'] = np.sin(2 * np.pi * df[column_name] / (5 * 24))
        df['cos_5day'] = np.cos(2 * np.pi * df[column_name] / (5 * 24))


        df['sin_90day'] = np.sin(2 * np.pi * df[column_name] / (90 * 24))
        df['cos_90day'] = np.cos(2 * np.pi * df[column_name] / (90 * 24))

        df['sin_120day'] = np.sin(2 * np.pi * df[column_name] / (120 * 24))
        df['cos_120day'] = np.cos(2 * np.pi * df[column_name] / (120 * 24))

        df['sin_180day'] = np.sin(2 * np.pi * df[column_name] / (180 * 24))
        df['cos_180day'] = np.cos(2 * np.pi * df[column_name] / (180 * 24))


        

    # Normalized cycle (assuming input is normalized to 1)
    df['sin_normal'] = np.sin(2 * np.pi * df[column_name])
    df['cos_normal'] = np.cos(2 * np.pi * df[column_name])

    return df

In [ ]:
data = pd.read_csv(train_path_)
data_test = pd.read_csv(test_path_)


data['event_id'] = data['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
data['event_idx'] = data.groupby('event_id', sort=False).ngroup()
data_test['event_id'] = data_test['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
data_test['event_idx'] = data_test.groupby('event_id', sort=False).ngroup()

data['event_t'] = data.groupby('event_id').cumcount()
data_test['event_t'] = data_test.groupby('event_id').cumcount()

In [ ]:
prefix = 'V9'
data = pd.read_parquet(os.path.join(data_dir, f"Simple_Train{prefix}_RAW.parquet"))
data_test = pd.read_parquet(os.path.join(data_dir,f"Simple_Test{prefix}_RAW.parquet"))

In [ ]:
data = calculate_sin_cos_cycles(data, 'event_t', ver_ = sin_version)
data_test = calculate_sin_cos_cycles(data_test, 'event_t', ver_ = sin_version)

In [ ]:
if enable_tabular_predictions:

    tr_ = pd.read_csv(f'IDSx_OOF_FromImages{tabular_targeted}.csv').drop(columns=['label']).rename(columns={'oof_pred': 'model_pred'})
    te_ = pd.read_csv(f'IDSx_TEST_FromImages{tabular_targeted}.csv').rename(columns={'preds': 'model_pred'})

    # tr_ = rw_[rw_['Type'] == 'Train'].drop(columns=['Type'])
    # te_ = rw_[rw_['Type'] == 'Test'].drop(columns=['Type'])

    display(tr_.head())
    display(te_.head())

    print(data.shape)
    data = pd.merge(left=data, right = tr_, on ='event_id', how='left')
    print(data.shape)

    print(data_test.shape)
    data_test = pd.merge(left=data_test, right = te_, on ='event_id', how='left')
    print(data_test.shape)
    data_test.head()


In [ ]:

# Features and target
X = data.drop(columns=['label', 'event_id', 'event_idx', 'event_t'])
y = data['label']
groups = data['event_id']


skf = StratifiedGroupKFold(n_splits=10, shuffle=False)



## Model Params 

In [ ]:
best_model_ = { 'max_depth': 10, 
                'num_leaves': 297, 
                'learning_rate': 0.012899582615551406, 
                'feature_fraction': 0.8543108407301498, 
                'bagging_fraction': 0.882130759492644, 
                'bagging_freq': 80, 
                'min_data_in_leaf': 6752, 
                'lambda_l1': 0.11023697527638228, 
                'lambda_l2': 3.494274960171233e-05, 
                'n_estimators': 11000, 
                'min_gain_to_split': 0.3815117888101758, 
                'scale_pos_weight': 3.1728213698029695, 
                'is_unbalanced': True,}


best_other_ = { 'init_score': 0.01}


lgb_params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',
            'boosting_type': 'gbdt',
            
            ## Best parameters from optuna
            'max_depth': best_model_['max_depth'],
            'num_leaves': best_model_['num_leaves'],
            'learning_rate': best_model_['learning_rate'],
            'feature_fraction': best_model_['feature_fraction'],
            'bagging_fraction': best_model_['bagging_fraction'],
            'bagging_freq': best_model_['bagging_freq'],
            'min_data_in_leaf': best_model_['min_data_in_leaf'],
            'lambda_l1': best_model_['lambda_l1'],
            'lambda_l2': best_model_['lambda_l2'],
            'n_estimators' : best_model_['n_estimators'],
            'min_gain_to_split': best_model_['min_gain_to_split'],

            
            'scale_pos_weight': best_model_['scale_pos_weight'],
            'is_unbalanced' : best_model_['is_unbalanced'],
            
            # Additional parameters for new LightGBM versions:
            'two_round': True,
            'num_threads': 64,
            'force_col_wise': True,
            'verbose': -1,
        }



In [ ]:
PREFIX_ = '29.2_SIN2_S24_R1_'
log_filename = 'local_log.txt'

## Run Training

In [ ]:
# To store the best models, logloss, and OOF predictions
lgb_models = []
lgb_logloss = []
oof_preds = np.zeros(len(X))  # Placeholder for OOF predictions

cnt = 0

# Cross-validation
for train_idx, valid_idx in skf.split(X, y, groups):
    print(f'\n Start training for fold: {cnt}')
    start_time = time.time()  # Start timer


    # Prepare the message including the current timestamp
    current_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
    message = f'\n\n\nV{PREFIX_} - No Images -{current_time} - Start training for fold: {cnt}\n'
    # Append the message to the log file
    with open(log_filename, 'a') as log_file:
        log_file.write(message)

    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    # train_data = lgb.Dataset(X_train, label=y_train)
    # valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

    train_init_score = np.full(X_train.shape[0], best_other_['init_score'])
    valid_init_score = np.full(X_valid.shape[0], best_other_['init_score'])
    train_data = lgb.Dataset(X_train, label=y_train, init_score=train_init_score)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data, init_score=valid_init_score)
    print("Custom Init_Score")

    model = lgb.train(
        lgb_params,
        train_data,
        num_boost_round = 3000,
        valid_sets=[valid_data],
        callbacks=[
                lgb.early_stopping(stopping_rounds=1_500, verbose=True),
            ]
        #early_stopping_rounds=500,  # Early stopping of 500
        #verbose_eval=False
    )

    end_time = time.time()  # End timer
    elapsed_time_minutes = (end_time - start_time) / 60  # Calculate elapsed time in minutes
    
    time_message = f"Total time for fold {cnt}: {elapsed_time_minutes:.2f} minutes"
    print(time_message)
    with open(log_filename, 'a') as log_file:
        log_file.write(time_message)

    # OOF predictions for the validation fold
    y_valid_pred = model.predict(X_valid)
    oof_preds[valid_idx] = y_valid_pred  # Store OOF predictions
    logloss = log_loss(y_valid, y_valid_pred)
    print(f"\nLogloss for LightGBM for fold: {cnt} is {logloss}")

    with open(log_filename, 'a') as log_file:
        log_file.write(f"\nLogloss for LightGBM for fold: {cnt} is {logloss}")
    lgb_logloss.append(logloss)

    lgb_models.append(model)
    cnt += 1

print(f"Mean Logloss for LightGBM: {np.mean(lgb_logloss)}")

with open(log_filename, 'a') as log_file:
        log_file.write(f"\nMean Logloss for LightGBM: {np.mean(lgb_logloss)} \n\n---------------------------------------------------- \n\n")

# Create a DataFrame for OOF predictions with `event_id`
oof_df = data[['event_id','event_idx','event_t']].copy()
oof_df['oof_pred'] = oof_preds

# Save the OOF predictions to a CSV file
oof_df.to_csv(f'{PREFIX_}oof_predictions.csv', index=False)
print("OOF predictions saved to 'oof_predictions.csv'.")

In [ ]:
print("Finished")

## Run Inference Predictions

In [ ]:
# Predicting on the test dataset using an ensemble of models
def predict_lgb(models, data_test):
    predictions = np.zeros(len(data_test))
    for model in models:
        predictions += model.predict(data_test)
    return predictions / len(models)

# Predict on the test set (replace 'data_test' with actual test data)
data_test_predictions_lgb = predict_lgb(lgb_models, data_test.drop(columns=['event_id', 'event_idx', 'event_t']))
data_test['label_lgbm'] = data_test_predictions_lgb


data_test['event_id'] = data_test['event_id']+'_X_'+data_test['event_t'].astype(str)


In [ ]:
## Save File of inference
data_test[['event_id', 'label_lgbm']].to_csv(f"V{PREFIX_}_submission_S{SEED_}_tbl_Heavy.csv",index=False)